In [1]:
import sys,os
sys.path.append(os.path.realpath('..'))

from scripts.functions import dilute, replenish,dilute_species, replenish_species, run_model,plot_result,plot_result_two_state
from scripts.species import DNA, RNA, Protein, Resource
from scripts.models import Repressor_model


import numpy as np
import scipy.integrate
import scipy.optimize
from bokeh.plotting import figure, output_file, show
import bokeh.plotting
import bokeh.io
from bokeh.models import Span


# We will use Matplotlib to make a 3D plot
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import panel as pn
pn.extension()

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
#######Define species #######

#T7 RNAP
T7_DNA = DNA()
T7_RNA = RNA()
T7 = Protein()

#Repressor
Repressor_DNA = DNA()
Repressor_RNA = RNA()
Repressor = Protein()

#GFP
GFP_DNA = DNA()
GFP_RNA = RNA()
GFP = Protein()

#Resource
R      = Resource()

#Others
TX = 1
TL = 1

#list of molecules
molecules_list = [T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,Repressor_DNA,Repressor_RNA,Repressor,R]


#give index num
for i in range(0,len(molecules_list)):
    molecules_list[i].idx = i




#define values 
##T7
T7_DNA.ic =0.01
T7_DNA.lc =0.01

T7.ic = 1
T7.lc = 1

##GFP
GFP_DNA.ic = 2
GFP_DNA.lc = 2

GFP.lc = 0

##Repressor 
Repressor_DNA.ic = 0.01
Repressor_DNA.lc = 0.01


#resources
R.ic =100
R.lc =100

#Define parameters
alpha=0.7;
beta=0.07;
K=1.0;
Kd = 0.5;
HC = 0.1;
KTL=10;
KTX=1;
lamb1=0;
lamb2=0;

parameters_list = (alpha,beta,K,Kd,KTL,KTX,lamb1,lamb2)

end_time = 30

end_cycle = end_time*4+1


In [3]:
#Positive control
T7_DNA.ic = 0
T7_DNA.lc = 0

#initial value
molecules_0 = [None]*len(molecules_list) #create empty list


for i in range(0,len(molecules_list)):
    molecules_0[i] = molecules_list[i].ic

#stage 1 kick start
result_all=[molecules_0] #initial condition


#kick start
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
replenish_list = [T7,R]
result_all,molecules_0 = run_model(Repressor_model,(0,4),parameters_list,molecules_0,dilution_list,replenish_list,result_all)


#self regeneration 
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
replenish_list = [T7,R]
result_all,molecules_0 = run_model(Repressor_model,(4,end_time),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



result_all = np.resize(result_all,(end_cycle,10))
for i in range(0,len(molecules_list)):
        (molecules_list[i]).pc_result = result_all.transpose()[i]

#T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,R = result_all.transpose()
plot_result(GFP.pc_result)

In [4]:
#Negative control
T7_DNA.ic = 0
T7_DNA.lc = 0

#initial value
molecules_0 = [None]*len(molecules_list) #create empty list


for i in range(0,len(molecules_list)):
    molecules_0[i] = molecules_list[i].ic

#stage 1 kick start
result_all=[molecules_0] #initial condition


#kick start
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
replenish_list = [T7,R]
result_all,molecules_0 = run_model(Repressor_model,(0,4),parameters_list,molecules_0,dilution_list,replenish_list,result_all)


#self regeneration 
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
replenish_list = [R]
result_all,molecules_0 = run_model(Repressor_model,(4,30),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



result_all = np.resize(result_all,(end_cycle,10))
for i in range(0,len(molecules_list)):
        (molecules_list[i]).nc_result = result_all.transpose()[i]

#T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,R = result_all.transpose()
plot_result(R.nc_result)

In [8]:
stored_data = []
for conc in np.logspace(-4,1, 100, endpoint=True):
    print(conc)
    #experiment 
    T7_DNA.ic = conc
    T7_DNA.lc = conc
    Repressor_DNA.ic = conc
    Repressor_DNA.lc = conc
    #initial value
    molecules_0 = [None]*len(molecules_list) #create empty list

    for i in range(0,len(molecules_list)):
        molecules_0[i] = molecules_list[i].ic

    #stage 1 kick start
    result_all=[molecules_0] #initial condition


    #DR = 0.2

    #kick start
    dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
    replenish_list = [T7,R]
    result_all,molecules_0 = run_model(Repressor_model,(0,4),parameters_list,molecules_0,dilution_list,replenish_list,result_all)


    #self regeneration 
    dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
    replenish_list = [R]
    result_all,molecules_0 = run_model(Repressor_model,(4,30),parameters_list,molecules_0,dilution_list,replenish_list,result_all)


    result_all = np.resize(result_all,(end_cycle,10))
    for i in range(0,len(molecules_list)):
            (molecules_list[i]).result = result_all.transpose()[i]

    #T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,R = result_all.transpose()
    #plot_result_two_state(GFP.result)
    #print((GFP).result[60]/GFP.pc_result[60])
    stored_data.append((GFP).result[100]/(GFP).pc_result[100])



0.0001
0.00011233240329780277
0.0001261856883066021
0.00014174741629268049
0.00015922827933410923
0.0001788649529057435
0.00020092330025650479
0.0002257019719633919
0.00025353644939701115
0.0002848035868435802
0.00031992671377973844
0.00035938136638046257
0.0004037017258596554
0.00045348785081285824
0.000509413801481638
0.0005722367659350221
0.0006428073117284319
0.0007220809018385464
0.0008111308307896872
0.0009111627561154887
0.001023531021899027
0.0011497569953977356
0.001291549665014884
0.0014508287784959402
0.0016297508346206436
0.0018307382802953678
0.0020565123083486517
0.002310129700083158
0.002595024211399737
0.002915053062825176
0.0032745491628777285
0.0036783797718286343
0.004132012400115334
0.004641588833612782
0.005214008287999685
0.005857020818056668
0.006579332246575682
0.007390722033525776
0.008302175681319744
0.0093260334688322
0.010476157527896652
0.011768119524349979
0.013219411484660288
0.01484968262254465
0.016681005372000592
0.01873817422860383
0.0210490414451202


In [9]:
conc = np.logspace(-4,1, 100, endpoint=True)

p = figure(width=800, height=400,x_axis_type = "log")
p.dot(conc,stored_data,size=20)
p.xaxis.axis_label = 'DNA concentration'
p.yaxis.axis_label = 'Normalized GFP'

show(p)
#bokeh.io.export_png(p, filename="plot.png")


In [7]:
print(conc)

[1.00000000e-04 1.12332403e-04 1.26185688e-04 1.41747416e-04
 1.59228279e-04 1.78864953e-04 2.00923300e-04 2.25701972e-04
 2.53536449e-04 2.84803587e-04 3.19926714e-04 3.59381366e-04
 4.03701726e-04 4.53487851e-04 5.09413801e-04 5.72236766e-04
 6.42807312e-04 7.22080902e-04 8.11130831e-04 9.11162756e-04
 1.02353102e-03 1.14975700e-03 1.29154967e-03 1.45082878e-03
 1.62975083e-03 1.83073828e-03 2.05651231e-03 2.31012970e-03
 2.59502421e-03 2.91505306e-03 3.27454916e-03 3.67837977e-03
 4.13201240e-03 4.64158883e-03 5.21400829e-03 5.85702082e-03
 6.57933225e-03 7.39072203e-03 8.30217568e-03 9.32603347e-03
 1.04761575e-02 1.17681195e-02 1.32194115e-02 1.48496826e-02
 1.66810054e-02 1.87381742e-02 2.10490414e-02 2.36448941e-02
 2.65608778e-02 2.98364724e-02 3.35160265e-02 3.76493581e-02
 4.22924287e-02 4.75081016e-02 5.33669923e-02 5.99484250e-02
 6.73415066e-02 7.56463328e-02 8.49753436e-02 9.54548457e-02
 1.07226722e-01 1.20450354e-01 1.35304777e-01 1.51991108e-01
 1.70735265e-01 1.917910